In [1]:
#import libraries
import os, sys
import json
import pandas as pd
#import geopandas as gpd
import numpy as np
from numpy import int64
import requests, io
import urllib.request
#import folium
#from folium import plugins
#import fiona
#import branca
#from branca.colormap import linear
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [6]:
url = 'https://www.worldometers.info/coronavirus/#countries'
response = requests.get(url)

In [12]:
countries_of_interest = ['Russia', 'UAE', 'Belarus', 'Kazakhstan', 'Lithuania', 'Latvia', 'Estonia', 'Uzbekistan', 'Cyprus']

In [38]:
columns_list = ['#', 'Country', 'Total cases', 'New cases', 'Total deaths', 'Total recov\'d', 'New recov\'d', 'Active']

In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [8]:
class HTMLTableParser:
    
    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):
            
            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)
                    
            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
                
        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
        
        return df

In [9]:
hp = HTMLTableParser()

In [10]:
table = hp.parse_url(url)[0][1] # Grabbing the table from the tuple
table.head(10)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,\nNorth America\n,"35,240,790","+6,632","806,677",+829,"26,864,936","+4,289","7,569,177","15,142",,,,,,North America,\n,,
1,,\nSouth America\n,"20,221,563",+879,"524,033",+19,"18,053,016","+1,088","1,644,514","19,850",,,,,,South America,\n,,
2,,\nAsia\n,"27,228,083","+5,838","418,598",+37,"25,270,089","+4,900","1,539,396","24,152",,,,,,Asia,\n,,
3,,\nEurope\n,"37,903,264","+3,537","885,126",+35,"26,902,383","+11,577","10,115,755","29,618",,,,,,Europe,\n,,
4,,\nAfrica\n,"4,157,953",,"110,745",,"3,717,254",,"329,954","2,666",,,,,,Africa,\n,,
5,,\nOceania\n,"54,582",+15,"1,117",,"34,482",+7,"18,983",9,,,,,,Australia/Oceania,\n,,
6,,\n\n,721,,15,,706,,0,0,,,,,,,\n,,
7,,World,"124,806,956","+16,901","2,746,311",+920,"100,842,866","+21,861","21,217,779","91,437","16,012",352.3,,,,All,\n,,
8,1,USA,"30,636,534",,"556,883",,"23,039,585",,"7,040,066","8,643","92,165","1,675","392,235,568","1,179,974","332,410,303",North America,11,597,1
9,2,Brazil,"12,136,615",,"298,843",,"10,601,658",,"1,236,114","8,318","56,805","1,399","28,600,000","133,861","213,654,630",South America,18,715,7


In [29]:
df = table.drop(table.index[[0,1,2,3,4,5,6,7]]).reset_index(drop=True)
df.head(10)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,1,USA,"30,636,534",,"556,883",,"23,039,585",,"7,040,066","8,643","92,165","1,675","392,235,568","1,179,974","332,410,303",North America,11,597,1
1,2,Brazil,"12,136,615",,"298,843",,"10,601,658",,"1,236,114","8,318","56,805","1,399","28,600,000","133,861","213,654,630",South America,18,715,7
2,3,India,"11,734,058",+464,"160,477",,"11,205,160","+2,144","368,421","8,944","8,443",115,"236,438,861","170,121","1,389,827,226",Asia,118,"8,661",6
3,4,Russia,"4,474,610",,"95,818",,"4,088,045",,"290,747","2,300","30,652",656,"117,900,000","807,645","145,979,943",Europe,33,"1,524",1
4,5,France,"4,313,073",,"92,908",,"283,507",,"3,936,658","4,634","65,971","1,421","60,773,434","929,564","65,378,416",Europe,15,704,1
5,6,UK,"4,307,304",,"126,284",,"3,712,658",,"468,362",748,"63,208","1,853","117,279,577","1,721,037","68,144,696",Europe,16,540,1
6,7,Italy,"3,419,616",,"105,879",,"2,753,083",,"560,654","3,546","56,619","1,753","47,395,288","784,727","60,397,160",Europe,18,570,1
7,8,Spain,"3,234,319",,"73,744",,"2,992,848",,"167,727","1,880","69,157","1,577","41,884,958","895,591","46,767,941",Europe,14,634,1
8,9,Turkey,"3,061,520",,"30,316",,"2,863,882",,"167,322","1,710","36,020",357,"36,768,437","432,591","84,995,840",Asia,28,"2,804",2
9,10,Germany,"2,689,205",,"75,708",,"2,445,300","+11,500","168,197","3,171","32,023",902,"47,578,793","566,560","83,978,444",Europe,31,"1,109",2


In [33]:
df.drop(df.tail(8).index,inplace=True)
df

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,1,USA,"30,636,534",,"556,883",,"23,039,585",,"7,040,066","8,643","92,165","1,675","392,235,568","1,179,974","332,410,303",North America,11,597,1
1,2,Brazil,"12,136,615",,"298,843",,"10,601,658",,"1,236,114","8,318","56,805","1,399","28,600,000","133,861","213,654,630",South America,18,715,7
2,3,India,"11,734,058",+464,"160,477",,"11,205,160","+2,144","368,421","8,944","8,443",115,"236,438,861","170,121","1,389,827,226",Asia,118,"8,661",6
3,4,Russia,"4,474,610",,"95,818",,"4,088,045",,"290,747","2,300","30,652",656,"117,900,000","807,645","145,979,943",Europe,33,"1,524",1
4,5,France,"4,313,073",,"92,908",,"283,507",,"3,936,658","4,634","65,971","1,421","60,773,434","929,564","65,378,416",Europe,15,704,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,217,Marshall Islands,4,,,,4,,0,,67,,,,"59,480",Australia/Oceania,"14,870",,
217,218,Samoa,3,,,,2,,1,,15,,,,"199,370",Australia/Oceania,"66,457",,
218,219,Vanuatu,3,,,,1,,2,,10,,470,"1,505","312,308",Australia/Oceania,"104,103",,664
219,220,Micronesia,1,,,,1,,0,,9,,,,"115,896",Australia/Oceania,"115,896",,


In [36]:
df_need = df[df['Country,Other'].isin(countries_of_interest)]

In [39]:
df_need.columns

Index(['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths',
       'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases',
       'Serious,Critical', 'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests',
       'Tests/\n1M pop\n', 'Population', 'Continent', '1 Caseevery X ppl',
       '1 Deathevery X ppl', '1 Testevery X ppl'],
      dtype='object')

In [129]:
df_need_col = df_need[['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases']]

In [130]:
for col in df_need_col.columns[2:8]: 
    df_need_col[col] = df_need_col[col].str.replace('+', '').str.replace(',', '').str.replace(' ', '').str.replace(' ', '')
    

<ipython-input-130-c6e31e82ebfa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_need_col[col] = df_need_col[col].str.replace('+', '').str.replace(',', '').str.replace(' ', '').str.replace(' ', '')


In [131]:
df_need_col.columns = columns_list

In [133]:
for col in df_need_col.columns[2:8]: 
    df_need_col[col] = pd.to_numeric(df_need_col[col], errors='ignore')

<ipython-input-133-2d43335ead01>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_need_col[col] = pd.to_numeric(df_need_col[col], errors='ignore')


In [134]:
df_need_col.fillna(0, inplace=True)
df_need_col.replace(np.nan, 0, inplace=True)
df_need_col.replace(np.inf, 0, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [136]:
df_need_col[columns_list[2:8]] = df_need_col[columns_list[2:8]].astype('int64')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [137]:
df_need_col

,#,Country,Total cases,New cases,Total deaths,Total recov'd,New recov'd,Active
3,4,Russia,4474610,0,95818,4088045,0,290747
40,41,UAE,444398,0,1451,427188,0,15759
47,48,Belarus,312474,0,2175,303030,0,7269
56,57,Kazakhstan,235095,1088,2934,212908,817,19253
63,64,Lithuania,210202,0,3501,195115,0,11586
84,85,Latvia,98094,0,1840,88322,0,7932
85,86,Estonia,97456,0,809,70346,0,26301
93,94,Uzbekistan,81678,119,623,80045,96,1010
101,102,Cyprus,42993,0,246,2057,0,40690
